# 半监督学习

> y_train 中未标记值必须为 -1

## SelfTrainingClassifier 自我训练

> base_estimator必须实现predict_proba

参数：criterion{‘threshold’, ‘k_best’}, default=’threshold’，
参数：threshold: default=0.75,当criterion=threshold时  predict_proba预测概率大于threshold时就把该样本分到对应的标签
参数：max_iter迭代次数（可能会有无标签的数据被抛弃）
参数: k_best:将具有最高预测概率的k_best个伪标签添加到数据集中。

## 标签传播
相似的数据应该具有相同的label。 构造相似矩阵

标签传播算法（Label Propagation Algorithm）是基于图的半监督学习方法，基本思路是从已标记的节点的标签信息来预测未标记的节点的标签信息，利用样本间的关系，建立完全图模型。

每个节点标签按相似度传播给相邻节点，在节点传播的每一步，每个节点根据相邻节点的标签来更新自己的标签，与该节点相似度越大，其相邻节点对其标注的影响权值越大，相似节点的标签越趋于一致，其标签就越容易传播。在标签传播过程中，保持已标记的数据的标签不变，使其将标签传给未标注的数据。最终当迭代结束时，相似节点的概率分布趋于相似，可以划分到一类中。

标签传播表示半监督图推理算法的几个变种，且它有如下2个特性：
- 可用于分类和回归任务
- 将数据投射到另一维空间的内核方法(Kernel methods )

值得注意的是，sklearn.semi_supervised中的半监督估计器能够利用这些额外的未标签数据来更好地捕捉到底层数据分布的形状(the shape of the underlying data distribution)，并对新的样本进行更好的泛化。当我们有极少量的有标签数据和大量的无标签数据时，半监督算法可以取得较好的表现。
scikit-learn提供了两种标签传播模型 --- LabelPropagation和LabelSpreading，这两种模型的工作原理是在输入数据集的所有项上构建一个相似度图(similarity graph)。
LabelPropagation和LabelSpreading有相同点，也存在明显的差异：

- LabelPropagation和LabelSpreading对图的相似度矩阵( similarity matrix)的修改，以及对标签分布的箝位效应( the clamping effect )。箝位允许算法在一定程度上改变真的标签数据的权重。LabelPropagation算法对输入标签进行硬箝位(hard clamping)，也就是a = 0。这个箝位系数可以放宽，可以说是a = 0.2，这意味着我们将始终保留80%的原始标签分布，但算法得到的置信度在20%以内。

- LabelPropagation使用从数据中构建的原始相似度矩阵（the raw similarity matrix），不做任何修改。相比之下，LabelSpreading将具有正则化特性的损失函数最小化，因此它通常对噪声更稳健（划重点）。该算法对原始图的修改版本进行迭代，并通过计算归一化的图Laplacian矩阵对边缘权重进行归一化。这个过程也被用于频谱聚类（Spectral clustering）中。

- LabelSpreading类似于基本的标签传播算法（The basic Label Propagation algorithm），但使用了基于归一化的graph Laplacian和soft clamping 的亲和矩阵（affinity matrix）在标签间进行传播。

标签传播模型有两种内置的内核方法。内核的选择对算法的可扩展性和性能都有影响。有以下2种方法可供选择。

- rbf，距离离的越近越接近于1，距离离的越远越接近于0，由关键字gamma指定。
- knn，找一个无标注的数据，然后取附近k个有标注的数据，无标注数据附近哪种标注的数据最多就取哪一个（以未标注的数据为圆心做knn，在指定范围内找到了有标注的数据，然后对未标注的数据进行打标，然后进行打标传播，直到未标注的数据全都标注以后，算法结束），由关键字n_neighbors指定。
rbf内核将产生一个完全连接的图，在内存中用一个密集矩阵表示。这个矩阵可能非常大，再加上算法每次迭代都要进行一次完整的矩阵乘法计算，会导致运行时间过长。另一方面，KNN 内核将产生一个更有利于内存的稀疏矩阵，可以大大减少运行时间。

> LabelSpreading有较好的抗噪性

### LabelPropagation

[图算法一览表](https://support.huaweicloud.com/usermanual-ges/ges_01_0031.html)

标签传播算法（Label Propagation）是一种基于图的半监督学习方法，其基本思路是用已标记节点的标签信息去预测未标记节点的标签信息。
利用样本间的关系建图，节点包括已标注和未标注数据，其边表示两个节点的相似度，节点的标签按相似度传递给其他节点。标签数据就像是一个源头，可以对无标签数据进行标注，节点的相似度越大，标签越容易传播。
（每个节点使用其邻居节点的标签中最多的标签来更新自身的标签。反复执行上述步骤，直到每个节点的标签都不再发生变化为止。）

> 高斯图模型
> 常用的图构建方法有knn图，也就是只保留每个节点的k近邻权重，其他的为0，也就是不存在边，因此是稀疏的相似矩阵。

[标签传播算法（Label Propagation）及Python实现](https://blog.csdn.net/zouxy09/article/details/49105265)

参数：kernel：{‘knn’, ‘rbf’} or callable, default=’rbf’
参数：gamma，高斯分布中的参数
参数：tol 收敛精度 ，max_iter 最大迭代次数
参数：n_neighbors：knn参数

### LabelSpreading
多了个alpha参数